In [1]:
import MDAnalysis
import numpy as np
import nglview as nv
from traitlets import observe
import copy 
import sys
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
#cell of function that parse instructions from file

def get_raw_instructions(file_of_instrictions):# function that get instructions information from file
    with open(file_of_instrictions, encoding = 'utf-8') as f:
        instr_arr = [el.split() for el in f.read().split(';')[:-1]]
        atom_arr = [np.array(el[-1].split(','), dtype='int') - 1 for el in instr_arr]
        res_instr = []
        for i in range(len(instr_arr)):
            res_instr.append(instr_arr[i][:-1] + [atom_arr[i].tolist()])
    return res_instr


def get_instructions(file_of_instrictions):# function that parce instructions information from file
    res_instr = get_raw_instructions(file_of_instrictions)
    n_jum = max([int(el[0]) for el in res_instr if el[0]!=-1])
    save_now_atom = [[[-1, []]] for _ in range(n_jum + 1)]
    array_of_instrictions = []
    for group, frame, atom_array in res_instr:
        group = int(group)
        save_now_atom[group].append([int(frame), atom_array])
        if len(save_now_atom[group]) >= 3:
            array_of_instrictions.append(copy.deepcopy(save_now_atom[group]))
            del save_now_atom[group][0]
    for i in range(len(save_now_atom)):
        array_of_instrictions.append(save_now_atom[i] + [[sys.maxsize, []]])
    array_of_instrictions.sort(key=lambda x:int(x[1][0]))
    return array_of_instrictions

In [3]:
#cell of function that implement instruction on view

def instr_ret(prew_frame, now_frame):# function that return index of instruction that need to be implement
    last_i = 0
    compare = lambda a,b: a!=b if prew_frame <= now_frame else True
    for i, el in enumerate(frames):
        if el > now_frame:
            return last_i
        elif compare(frames[last_i], frames[i]):
            last_i = i
    return last_i

def visual_atoms(instruct):# function that implement instruction on view
    global now_showing
    if now_showing != []:
        view.remove_distance()
        view.remove_distance()
        view.add_representation(repr_type='ball+stick', selection=now_showing)
    now_showing = []
    hide, show_now, show_futer = instruct
    atoms = show_now[1]
    [now_showing.append(el) for el in atoms]
    O_now = atoms[0]
    H_arr = atoms[1:]
    view.add_representation(repr_type='ball+stick', selection=[O_now], color=0xfa0202)
    view.add_representation(repr_type='ball+stick', selection=H_arr, color=0x2003fc)
    if show_futer[0] != sys.maxsize:
        atoms = show_futer[1]
        O_futer = atoms[0]
        H_arr = atoms[1:-1]
        jump_H = atoms[-1]
        view.add_representation(repr_type='ball+stick', selection=[O_futer], color=0x961212)
        view.add_representation(repr_type='ball+stick', selection=H_arr, color=0x12085c)
        view.add_representation(repr_type='ball+stick', selection=[jump_H], color=0x03a5fc)
        [now_showing.append(el) for el in atoms]
        sele_jump_H = '@' + str(jump_H)
        sele_O_now = '@' + str(O_now)
        sele_O_futer = '@' + str(O_futer)
        view.add_distance(atom_pair=[[sele_O_now, sele_jump_H]], \
                          label_color="black", label_size=1.0, color="green")
        view.add_distance(atom_pair=[[sele_O_futer, sele_jump_H]], \
                          label_color="black", label_size=1.0, color="green")
        view.center(duration=0, selection=sele_jump_H)
        view.control.zoom(-0.4)
    else:
        sele = '@'+str(O_now)
        view.center(duration=0, selection=sele)
        view.control.zoom(-0.4)

In [4]:
def on_frame_change(change): #observer function that change view if it is necessary
    global view
    global comand_frame
    global direct
    global prev_frame
    global i_now
    zoom_scale = -0.5
    instrict_i = instr_ret(prev_frame, view.frame)
    instrict = array_of_instrictions[instrict_i]
    comand_frame = instrict[1][0]
    now_direct = 1 if prev_frame <= view.frame else -1
    i_now = [] if direct != now_direct else i_now
    direct = now_direct
    while (instrict[1][0] == comand_frame) and (instrict_i not in i_now):
        i_now.append(instrict_i)
        visual_atoms(instrict)
        instrict_i += 1
        if instrict_i < 0: instrict_i = 0; break;
        if instrict_i > len(frames) - 1: instrict_i = len(frames) - 1; break;
        instrict = array_of_instrictions[instrict_i]
    prev_frame = view.frame

In [5]:
#cell set file name
traj_file_name = '' # you can write file name in code
if traj_file_name == '': # or from keybord
    traj_file_name = input("Pleas write trajectory file name (or press Enter to use default file):")
if traj_file_name == '':
    traj_file_name = "/Users/danilezov/Downloads/res_1/dump_s.dmp.LAMMPSDUMP"
instrictions_file_name = '' # you can write file name in code
if instrictions_file_name == '': # or from keybord
    instrictions_file_name = input("Pleas write instrictions file name (or press Enter to use default file):")
if instrictions_file_name == '':
    instrictions_file_name = "/Users/danilezov/Downloads/res_1/vis_mod.txt"

Pleas write trajectory file name (or press Enter to use default file): 
Pleas write instrictions file name (or press Enter to use default file): 


In [6]:
#cell set global parameters

array_of_instrictions = get_instructions(instrictions_file_name)
i = 1
prev_frame = 0
i_now = []
direct = 1
now_showing = []
frames = [int(el[1][0]) for el in array_of_instrictions]

In [7]:
#make view from file
u = MDAnalysis.Universe(traj_file_name)
view = nv.show_mdanalysis(u, default = False)
view.add_representation('ball+stick', selection='all')
view.observe(on_frame_change, names=['frame'])
view.center()
view.control.zoom(0.15)
view

NGLWidget(max_frame=139)

<h2>Legend:</h2>

Curent hydronium:
- ![#fa0202](https://via.placeholder.com/15/fa0202/000000?text=+) `Oxygen`
- ![#2003fc](https://via.placeholder.com/15/2003fc/000000?text=+) `Hydrogen`
- ![#03a5fc](https://via.placeholder.com/15/03a5fc/000000?text=+) `Hydrogen that will go to another H2O and make hydronium`

Futer hydronium:
- ![#961212](https://via.placeholder.com/15/961212/000000?text=+) `Oxygen`
- ![#12085c](https://via.placeholder.com/15/12085c/000000?text=+) `Hydrogen`